In [18]:
import xml.etree.ElementTree as ET
import os


def write_xml(tree, output_file):
    """Writes XML tree to a file."""
    tree.write(output_file, encoding="utf-8", xml_declaration=True)


def remove_namespace(element):
    """Removes namespace from an XML element."""
    # Remove namespace in element tag
    if "}" in element.tag:
        element.tag = element.tag.split("}", 1)[1]

    # Remove namespace in element attributes
    new_attrib = {}
    for name, value in element.attrib.items():
        if "}" in name:
            new_name = name.split("}", 1)[1]
            new_attrib[new_name] = value
        else:
            new_attrib[name] = value
    element.attrib = new_attrib

    # Apply function for each child element
    for child in element:
        remove_namespace(child)


# Set the path to the XML file and output file
path_to_file = 'files\AF_Safety_OG.xml'
path_to_output = 'files_out\AF_Safety_OG.xml'

# Check if folder exists if not, create it
if not os.path.exists(os.path.dirname(path_to_output)):
    os.makedirs(os.path.dirname(path_to_output))

# Load the XML data
tree = ET.parse(path_to_file)
root = tree.getroot()


# Find the 'Engineering' element and update its 'version' attribute
for elem in root.findall('Engineering'):
    if 'version' in elem.attrib and elem.attrib['version'] == 'V17':
        elem.set('version', 'V16')

# Find the 'Sections' element under 'Interface' and update its 'xmlns' attribute
for elem in root.findall('./Interface/Sections'):
    xmlns_attr = '{http://www.w3.org/2001/XMLSchema-instance}schemaLocation'
    if xmlns_attr in elem.attrib:
        old_value = elem.attrib[xmlns_attr]
        new_value = old_value.replace('v5', 'v4')
        elem.set(xmlns_attr, new_value)

# Call the namespace removal function
remove_namespace(root)

# Save the cleaned XML data to a new file
write_xml(tree, path_to_output)